# GLAD Alert widgets

In [6]:
#Import Global Metadata etc

%run '0.Importable_Globals.ipynb'

https://production-api.globalforestwatch.org/v1/glad-alerts/admin/COD?period=2017-01-20%2C2017-12-26&thresh=30&_=1516027383380
    
    
https://production-api.globalforestwatch.org/v1/glad-alerts/admin/COD/4?period=2017-01-20%2C2017-12-26&thresh=30&_=1516027383381
    
    
https://production-api.globalforestwatch.org/v1/glad-alerts?geostore=40c6e39d84c6fb3c5e3781cc69fe7444&period=2017-10-01%2C2017-12-26&thresh=30&_=1516027383379
